##### Project: Opioid Exposed Infant Covariates
##### Investigator: Stephen Patrick, Sarah Loch
##### Programmers: Sander Su, Chris Guardo
##### Date Created: 01/17/23
##### Last Modified: 09/30/25

#### Notes: 
This step comes after birth visit in the workspace pipeline. It matches infant reccords to death reccords available through OMOP tables

In [0]:
%run "../Project_modules"

##### Check if 41K infants are in OMOP DEATH table and output MRN, DOB, Visit start, visit end, DEATH Columns + concept names, Flag for if death date is within birth hospitalization stay + 3 days

In [0]:
phenotype_table_location = " ***Insert file location*** "
phenotype_table=spark.sql(f"SELECT * FROM {phenotype_table_location}")

##### cohort:  infant in phenotype cohort with exposed to opioid

In [0]:
phenotype_cohort = get_phenotype_cohort(phenotype_table)
phenotype_cohort.createOrReplaceTempView("phenotype_cohort")

##### To See if the infants in the DEATH table (V_DEATH)

In [0]:
sql=f"""
      select * from {death_table} WHERE person_id in (select distinct baby_person_id from phenotype_cohort);
     """
infant_death = spark.sql(sql)
infant_death.name='infant_death'
register_parquet_global_view(infant_death)

##### Validation

In [0]:
sql="""
       select count(*) as total,count(distinct person_id) as unique_baby from global_temp.infant_death;

   """
inspect_df= spark.sql(sql)
inspect_df.display()


##### Add phenotype information for review
##### FLAG if the Death happened during birth hospitalization

In [0]:

sql=f"""
      select BABY_PERSON_SOURCE_VALUE as baby_mrn,DEATH_DATE,DEATH_DATETIME,DEATH_TYPE_CONCEPT_ID,CAUSE_CONCEPT_ID,CAUSE_SOURCE_VALUE,
      CAUSE_SOURCE_CONCEPT_ID,
      case when death_date is not null and death_date >= first_visit_start_date and death_date <= first_visit_end_date then 1 
      when death_date is null then null else 0 end as if_within_bh,
      BIRTH_DATETIME,FIRST_VISIT_START_DATE,FIRST_VISIT_START_DATETIME,FIRST_VISIT_END_DATE,
      b.BABY_1ST_VISIT_PROBLEM as BABY_1ST_VISIT_PROBLEM_DETAIL,
      MOM_PERSON_SOURCE_VALUE as mom_mrn,AGE_AT_DELIVERY,MOM_BIRTH_DATETIME,MOM_GENDER,MOM_RACE,BABY_PERSON_SOURCE_VALUE,
      BABY_BIRTH_DATETIME,BABY_GENDER,BABY_RACE,
      LIVE_BIRTH_CODE,PREGNANCY_CODE,GESTATIONAL_AGE_W33_OR_UNCERTAIN,
      CRITICAL_ILLNESS_4CPT,RESPIRATORY_PROCEDURE_CODE,FETAL_ANOMALIES_CODE,GESTATIONAL_AGE_UNCERTAIN,
      NOWS_BABY_CODE,INFANT_TOX_LAB,MOM_OUD,MOM_OUD_INPATIENT,MOM_OUD_OUTPATIENT,MOM_DRUG,MOM_DRUG_IN_NOTE,
      MOM_OPIOID_TOX,c.BABY_1ST_VISIT_PROBLEM 
   
      from global_temp.infant_death a 
      left join global_temp.mom_baby_step1_baby1stvisit b on a.person_id = b.baby_person_id
      inner join {phenotype_table} c 
      on a.person_id = c.baby_person_id;
   """
infant_death_all = spark.sql(sql)
infant_death_all.name='infant_death_all'
register_parquet_global_view(infant_death_all)

##### Validation

In [0]:
sql="""
       select count(*) as total,count(distinct baby_mrn) as unique_baby from global_temp.infant_death_all;
    """
inspect_df = spark.sql(sql)
inspect_df.display()

### Save Output for future use

In [0]:
infant_death_all.write.mode("overwrite").saveAsTable(f"covariate_output.infant_death_all")